In [1]:
# Importing some shizzle.

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import time
import requests
import datetime
import sys
import os
import pathlib
import seaborn as sns
from sklearn.metrics import roc_auc_score

location = 'rjsietsma'

read_loc = '/home/'+location+'/Documents/School/Master_DSLS/Final_Thesis/Initial_Data_exploration/'

In [2]:
# Define function to perform Shapiro-Wilk, Kolmogorov-Smirnov, Wilcoxon-/Mann-Whitney U -test, and a pearson correlation test.

def perform_stats(x, y):
    shapiro_x = stats.shapiro(x)[1]
    shapiro_y = stats.shapiro(y)[1]
    if shapiro_x >= 0.05:
        print("X is likely normally distributed.")
    else:
        print("X is likely not normally distributed.")
    if shapiro_y >= 0.05:
        print("Y is likely normally distributed.")
    else:
        print("Y is likely not normally distributed.")
    try:
        p_ks_xy = stats.ks_2samp(x, y)[1]
        print(f"The Kolmogorov-Smirnov test p-value: {p_ks_xy}")
    except Exception:
        print("Kolmogorov-Smirnov could not be performed!")
    try:
        p_wc_xy = stats.wilcoxon(x, y)[1]
        print(f"The Wilcoxon test p-value: {p_wc_xy}")
    except ValueError:
        p_mw_xy = stats.mannwhitneyu(x, y)[1]
        print(f"Wilcoxon could not be performed, \n"
             f"Using Mann-Whitney rank test p-value: {p_mw_xy}")
    except Exception:
        print("Neither Wilcoxon nor Mann-Whitney tests could be performed!")
    try:
        p_pears_xy = stats.pearsonr(x, y)
        print(f"The Pearson correlation: {p_pears_xy[0]},\n"
             f"p-value: {p_pears_xy[1]}")
    except Exception:
        print("Pearson correlation could not be performed!")

# Define function to calculate the Z-scores of given data.

def calc_z_scores(data):
    centered = data - data.mean(axis=0)
    return centered / centered.std(axis=0)

In [3]:
# Read in the data
full_snv_dataset = pd.read_csv(read_loc+'full_dataset.csv')
full_snv_dataset

,chr,pos,ref,alt,label,gene,consequence,capice,source
0,14,68196054,GCCCTG,G,LP/P,RDH12,FRAME_SHIFT,0.988552,train
1,20,10626717,TCA,T,LP/P,JAG1,STOP_GAINED,0.990329,train
2,20,10625898,CTG,C,LP/P,JAG1,FRAME_SHIFT,0.991696,train
3,20,10628741,AC,A,LP/P,JAG1,FRAME_SHIFT,0.983607,train
4,20,10625509,ACT,A,LP/P,JAG1,DOWNSTREAM,0.987999,train
...,...,...,...,...,...,...,...,...,...
456004,17,29556342,G,A,LP/P,NF1,SYNONYMOUS,0.000139,test
456005,11,5248177,A,T,LP/P,HBB,SYNONYMOUS,0.069934,test
456006,15,48787324,T,C,LP/P,FBN1,SYNONYMOUS,0.909190,test
456007,19,17947957,G,A,LP/P,JAK3,SYNONYMOUS,0.001238,test


In [8]:
full_mapped_ds = pd.read_csv(read_loc+'optimal_f1_full_ds_v2.csv', header=0)
full_mapped_ds.sort_values(by='default_f1', ascending=False)

,gene,default_auc,default_f1,default_recall,default_fpr,default_spec,optimal_c,optimal_auc,optimal_f1,optimal_recall,optimal_fpr,optimal_spec,n_tot,n_benign,n_malign
2775,PRKN,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,1.0,0.0,1.0,11,7,4
1659,BUB1B,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,1.0,0.0,1.0,73,66,7
2102,GALNT3,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,1.0,0.0,1.0,12,10,2
1655,PNPLA1,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,0.8,0.2,0.8,31,26,5
2782,TMEM80,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,1.0,0.0,1.0,5,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2625,C10orf2,0.469697,0.0,0.0,1.0,0.0,0.020,0.469697,0.000000,0.0,1.0,0.0,133,132,1
2330,SPTLC2,0.455157,0.0,0.0,1.0,0.0,0.011,0.750374,0.170213,0.0,1.0,0.0,229,223,6
2615,PLCD1,0.500000,0.0,0.0,1.0,0.0,0.020,0.500000,0.000000,0.0,1.0,0.0,8,7,1
2611,UBE2A,0.500000,0.0,0.0,1.0,0.0,0.020,0.500000,0.000000,0.0,1.0,0.0,3,2,1


In [15]:
filter1 = full_mapped_ds['n_benign'] > 5
filter2 = full_mapped_ds['n_malign'] > 5

full_mapped_ds.where(filter1 & filter2).dropna().sort_values(by='default_f1', ascending=False).tail(10)

,gene,default_auc,default_f1,default_recall,default_fpr,default_spec,optimal_c,optimal_auc,optimal_f1,optimal_recall,optimal_fpr,optimal_spec,n_tot,n_benign,n_malign
197,C2orf71,0.779630,0.208333,0.833333,0.166667,0.833333,0.024,0.787037,0.217391,0.833333,0.166667,0.833333,141.0,135.0,6.0
516,CLCN2,0.833333,0.205882,1.000000,0.000000,1.000000,0.033,0.891975,0.285714,0.857143,0.142857,0.857143,169.0,162.0,7.0
1135,PDE11A,0.704272,0.196721,0.857143,0.142857,0.857143,0.035,0.807076,0.307692,0.857143,0.142857,0.857143,114.0,107.0,7.0
2258,MYH11,0.871699,0.188679,0.833333,0.166667,0.833333,0.023,0.874911,0.200000,0.833333,0.166667,0.833333,473.0,467.0,6.0
2006,SCN3A,0.748756,0.181818,0.666667,0.333333,0.666667,0.035,0.786070,0.275862,0.666667,0.333333,0.666667,207.0,201.0,6.0
954,IFIH1,0.753148,0.147368,0.636364,0.363636,0.636364,0.045,0.792844,0.291667,0.636364,0.363636,0.636364,603.0,592.0,11.0
2455,GARS,0.561587,0.120000,0.230769,0.769231,0.230769,0.016,0.630599,0.175439,0.230769,0.769231,0.230769,329.0,316.0,13.0
2399,MUT,0.578992,0.075949,0.428571,0.571429,0.428571,0.010,0.724650,0.102564,0.428571,0.571429,0.428571,262.0,255.0,7.0
2376,SCARB1,0.487032,0.000000,0.000000,1.000000,0.000000,0.020,0.487032,0.000000,0.000000,1.000000,0.000000,355.0,347.0,8.0
2330,SPTLC2,0.455157,0.000000,0.000000,1.000000,0.000000,0.011,0.750374,0.170213,0.000000,1.000000,0.000000,229.0,223.0,6.0


In [26]:
for consequence in full_snv_dataset['consequence'].unique():
    print(f'Consequence: {consequence}')
    print(full_snv_dataset[full_snv_dataset['consequence'] == consequence]['label'].value_counts())
    

Consequence: FRAME_SHIFT
LP/P    17804
LB/B     8069
Name: label, dtype: int64
Consequence: STOP_GAINED
LP/P    14842
LB/B     4458
Name: label, dtype: int64
Consequence: DOWNSTREAM
LB/B    22319
LP/P     1224
Name: label, dtype: int64
Consequence: CANONICAL_SPLICE
LP/P    5788
LB/B    2817
Name: label, dtype: int64
Consequence: INTRONIC
LB/B    61132
LP/P      620
Name: label, dtype: int64
Consequence: INFRAME
LB/B    4053
LP/P     824
Name: label, dtype: int64
Consequence: SPLICE_SITE
LB/B    8092
LP/P     839
Name: label, dtype: int64
Consequence: REGULATORY
LB/B    13540
LP/P      874
Name: label, dtype: int64
Consequence: 3PRIME_UTR
LB/B    688
LP/P     11
Name: label, dtype: int64
Consequence: UPSTREAM
LB/B    14500
LP/P      958
Name: label, dtype: int64
Consequence: NONCODING-CHANGE
LB/B    1113
LP/P       2
Name: label, dtype: int64
Consequence: NON_SYNONYMOUS
LB/B    62811
LP/P    19845
Name: label, dtype: int64
Consequence: NONCODING_CHANGE
LB/B    592
LP/P     82
Name: labe

In [25]:
full_snv_dataset[full_snv_dataset['consequence'] == consequence]['label'].value_counts()['LB/B']

1

# Notes meeting 24-03-2020
- Get all genes that perform bad from the histograms earlier in this notebook
- Add FPR, recall, sensitivity and specificity to mapper function
- Add vkgl and test data to optimum AUC mapper
- *see school bookmark for gene panels* calculate auc and compare auc's
    - Immunodeficiency (big chunk)
    - Developmental delay
    - 5GPM everything except late onset, but is very large
    - More popular panels will have more information, possibly easier to interpret.
 - Phase 1:
     - Make a list of biases and other easy to test statistics per gene / chromosome
     - Grab genes that are difficult to predict and label them.
     - Apply (NaN)GMLVQ on (to predict) to show correlations / causations.


[google docs](https://docs.google.com/document/d/1D5SiNbeDEfY2hTWquS88MGUrLv6IgZzfCzflppRGb5k/edit) of what is interesting

To add onto the idea of novel annotations, below are some of the ideas I find interesting:
- pLI (loss of intolerance)
- Popmax filtering AF (we might be already using it in benchmarking but I guess there is no harm in explicitly testing it?)
- [pext](https://www.biorxiv.org/content/10.1101/554444v1)
- [Enhancer features](https://www.sciencedirect.com/science/article/abs/pii/S0002929720300124)
- Features included in this [paper](https://www.nature.com/articles/s41467-019-13212-3)

Interesting papers:

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6369448/

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5618255/ (supplement 5)



# Notes on training data on GCC cluster
- Some gene panels seem to be included
    - Allergy/Immunology/Infectious
    - Audiologic/Otolaryngologic
    - Biochemical
    - Cardiovascular
    - Craniofacial (facial bones)
    - Dermatologic
    - Endocrine
    - Gastrointestinal
    - Genitourinary
    - Hematologic
    - Musculoskeletal
    - Neurologic
    - Obstetric
    - Oncologic
    - Ophthalmologic
    - Pulmonary
    - Renal
        - Documentation is key, that shows when I looked up the training script.
        - CAPICE on github could use some per method documentation.
- Some really unclear columns are in the real training data:
    - General
    - inTest (is just False)
- Lots of commented out lines of code.

From the umcg-sli/variant_prioritization/data:
    - train_results.txt seems to be the same, somewhat.
    - train.txt doesn't seem at all what I have.
    - test_results seems to be the same somewhat.

Looking in past-paper:
    - test_results is a bit more detailed on cluster.
    - Same for train_results.
    - train.txt is the same as in /data.
    
Take home note: Ask Shuang what needs to happen if I want to retrain the model from scratch.

# Investigate:
- What was used in CADD to make that top of the line at that time?
    - Or just CADD just a search engine that searches the Alt through different pathogenicity predictors?

- CADD already takes into account the phylogenetic tree.
    - But not how new a gene is within the human race
- CADD is a logistic regression model
- 
    
